<a href="https://colab.research.google.com/github/Gladiator07/Deep-Learning-with-PyTorch/blob/main/Linear_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch has a whole submodule dedicated to neural networks, called torch.nn. It contains the building blocks needed to create all sorts of neural network architectures. Those building blocks are called modules in PyTorch parlance (such building blocks are often referred to as layers in other frameworks). A PyTorch module is a Python class deriving from the nn.Module base class. A module can have one or more Parameter instances as attributes, which are tensors whose values are optimized during the training process (think w and b in our linear model). A module can also have one or more submodules (subclasses of nn.Module) as attributes, and it will be able to track their parameters as well.

NOTE: The submodules must be top-level attributes, not buried inside list or dict instances! Otherwise, the optimizer will not be able to locate the submodules (and, hence, their parameters). For situations where your model requires a list or dict of submodules, PyTorch provides nn.ModuleList and nn.ModuleDict. 


Unsurprisingly, we can find a subclass of nn.Module called nn.Linear, which applies an affine transformation to its input (via the parameter attributes weight and bias) and is equivalent to what we implemented earlier in our thermometer experiments. We’ll now start precisely where we left off and convert our previous code to a form that uses nn.

In [ ]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)

In [ ]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [ ]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 3, 10,  9,  7,  8,  6,  5,  0,  1]), tensor([4, 2]))

In [ ]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

The constructor to nn.Linear accepts three arguments: the number of input features, the number of output features, and whether the linear model includes a bias or not (defaulting to True, here):

In [ ]:
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[0.3387],
        [0.3465]], grad_fn=<AddmmBackward>)

The number of features in our case just refers to the size of the input and the output tensor for the module, so 1 and 1. If we used both temperature and barometric pressure as input, for instance, we would have two features in input and one feature in output. As we will see, for more complex models with several intermediate modules, the number of features will be associated with the capacity of the model.

In [ ]:
linear_model.weight

Parameter containing:
tensor([[0.0411]], requires_grad=True)

In [ ]:
linear_model.bias

Parameter containing:
tensor([0.1076], requires_grad=True)

In [ ]:
x = torch.ones(1)
linear_model(x)

tensor([0.1486], grad_fn=<AddBackward0>)

Although PyTorch lets us get away with it, we don’t actually provide an input with the right dimensionality. We have a model that takes one input and produces one output, but PyTorch nn.Module and its subclasses are designed to do so on multiple samples at the same time. To accommodate multiple samples, modules expect the zeroth dimension of the input to be the number of samples in the batch.

## Batching Inputs

In [ ]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486],
        [0.1486]], grad_fn=<AddmmBackward>)

In [ ]:
t_u.shape

torch.Size([11, 1])

In [ ]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

In [ ]:
linear_model.parameters()

<generator object Module.parameters at 0x7f84d1c561a8>

In [ ]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.9165]], requires_grad=True), Parameter containing:
 tensor([-0.6471], requires_grad=True)]

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), # <1>
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 272.6239, Validation loss 268.3783
Epoch 1000, Training loss 3.5114, Validation loss 2.5710
Epoch 2000, Training loss 3.0428, Validation loss 2.5042
Epoch 3000, Training loss 3.0355, Validation loss 2.4961

Parameter containing:
tensor([[5.3721]], requires_grad=True)
Parameter containing:
tensor([-17.2292], requires_grad=True)
